
Tensorflow didn't work out of the box on MacOS with python due to https://github.com/python-poetry/poetry/issues/8271 so I had to also add the tensorflow-macos package

In [57]:
import tensorflow as tf
from tensorflow import keras

model = keras.models.load_model('bees-wasps.h5')

converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()

with open('bees-wasps.tflite', 'wb') as f_out:
    f_out.write(tflite_model)

INFO:tensorflow:Assets written to: /var/folders/0w/1wznyjl95gvd6tj6ltw5vc240000gn/T/tmp4q81rnul/assets


INFO:tensorflow:Assets written to: /var/folders/0w/1wznyjl95gvd6tj6ltw5vc240000gn/T/tmp4q81rnul/assets
2023-11-26 00:29:23.550390: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2023-11-26 00:29:23.550524: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2023-11-26 00:29:23.552122: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /var/folders/0w/1wznyjl95gvd6tj6ltw5vc240000gn/T/tmp4q81rnul
2023-11-26 00:29:23.552738: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2023-11-26 00:29:23.552742: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /var/folders/0w/1wznyjl95gvd6tj6ltw5vc240000gn/T/tmp4q81rnul
2023-11-26 00:29:23.559210: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2023-11-26 00:29:23.708704: I tensorflow/cc/saved_model/loader.cc:217] Running initialization

1. What's the size of the converted model?

In [58]:
# get the size of the model
import os
os.path.getsize('bees-wasps.tflite') / 1024 / 1024

42.787742614746094

2. To be able to use this model, we need to know the index of the input and the index of the output.

What's the output index for this model?

In [59]:
import tensorflow.lite as tflite
interpreter = tflite.Interpreter(model_path='bees-wasps.tflite')
interpreter.allocate_tensors()

output_index = interpreter.get_output_details()[0]['index']
output_index

13

In [60]:
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

resize the image to 150x150 based on previous homework

In [61]:
url = 'https://habrastorage.org/webt/rt/d9/dh/rtd9dhsmhwrdezeldzoqgijdg8a.jpeg'
image = download_image(url)
image = prepare_image(image, target_size=(150, 150))



Now we need to turn the image into numpy array and pre-process it.

Tip: Check the previous homework. What was the pre-processing we did there?

After the pre-processing, what's the value in the first pixel, the R channel?

In [67]:
import numpy as np
def preprocess_input(x):
    # rescale by 1/255
    return x / 255.0

x = np.array(image, dtype='float32')
X = np.array([x])

X = preprocess_input(X)
X[0][0][0]

array([0.94509804, 0.9098039 , 0.8509804 ], dtype=float32)

Now let's apply this model to this image. What's the output of the model?

In [63]:
input_index = interpreter.get_input_details()[0]['index']
interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)
preds

array([[0.65921456]], dtype=float32)

In [64]:
!docker pull agrigorev/zoomcamp-bees-wasps:v2
!docker images agrigorev/zoomcamp-bees-wasps:v2

v2: Pulling from agrigorev/zoomcamp-bees-wasps
Digest: sha256:823f8536a45968f40ee3daf8a2da030b914912a382a4611610b3b84d36d2924c
Status: Image is up to date for agrigorev/zoomcamp-bees-wasps:v2
docker.io/agrigorev/zoomcamp-bees-wasps:v2
REPOSITORY                      TAG       IMAGE ID       CREATED      SIZE
agrigorev/zoomcamp-bees-wasps   v2        b9f6c13de368   8 days ago   662MB


In [72]:
import requests

url = 'http://localhost:8080/2015-03-31/functions/function/invocations'

data = {'url': 'https://habrastorage.org/webt/rt/d9/dh/rtd9dhsmhwrdezeldzoqgijdg8a.jpeg'}

result = requests.post(url, json=data).json()
print(result)

[0.44533517956733704]
